In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# Mistral 베이스 모델과 Instruct 모델 로드
long_context_model_name = "winglian/Llama-3-8b-64k-PoSE" # 롱 컨텍
base_model_name = "kuotient/Meta-Llama-3-8B"
target_model_name = "~~"  # target 모델 이름 지정
long_context_model = AutoModelForCausalLM.from_pretrained(long_context_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

def calculate_weight_diff(base_weight, chat_weight):
    return torch.abs(base_weight - chat_weight).mean().item()

def calculate_layer_diffs(long_context_model, base_model):
    layer_diffs = []
    for base_layer, chat_layer in zip(long_context_model.model.layers, base_model.model.layers):
        layer_diff = {
            'input_layernorm': calculate_weight_diff(base_layer.input_layernorm.weight, chat_layer.input_layernorm.weight),
            'mlp_down_proj': calculate_weight_diff(base_layer.mlp.down_proj.weight, chat_layer.mlp.down_proj.weight),
            'mlp_gate_proj': calculate_weight_diff(base_layer.mlp.gate_proj.weight, chat_layer.mlp.gate_proj.weight),
            'mlp_up_proj': calculate_weight_diff(base_layer.mlp.up_proj.weight, chat_layer.mlp.up_proj.weight),
            'post_attention_layernorm': calculate_weight_diff(base_layer.post_attention_layernorm.weight, chat_layer.post_attention_layernorm.weight),
            'self_attn_q_proj': calculate_weight_diff(base_layer.self_attn.q_proj.weight, chat_layer.self_attn.q_proj.weight),
            'self_attn_k_proj': calculate_weight_diff(base_layer.self_attn.k_proj.weight, chat_layer.self_attn.k_proj.weight),
            'self_attn_v_proj': calculate_weight_diff(base_layer.self_attn.v_proj.weight, chat_layer.self_attn.v_proj.weight),
            'self_attn_o_proj': calculate_weight_diff(base_layer.self_attn.o_proj.weight, chat_layer.self_attn.o_proj.weight)
        }
        layer_diffs.append(layer_diff)
    return layer_diffs

def apply_layer_diffs(target_model, layer_diffs):
    for layer, layer_diff in zip(target_model.model.layers, layer_diffs):
        layer.input_layernorm.weight.data += layer_diff['input_layernorm']
        layer.mlp.down_proj.weight.data += layer_diff['mlp_down_proj']
        layer.mlp.gate_proj.weight.data += layer_diff['mlp_gate_proj']
        layer.mlp.up_proj.weight.data += layer_diff['mlp_up_proj']
        layer.post_attention_layernorm.weight.data += layer_diff['post_attention_layernorm']
        layer.self_attn.q_proj.weight.data += layer_diff['self_attn_q_proj']
        layer.self_attn.k_proj.weight.data += layer_diff['self_attn_k_proj']
        layer.self_attn.v_proj.weight.data += layer_diff['self_attn_v_proj']
        layer.self_attn.o_proj.weight.data += layer_diff['self_attn_o_proj']

def visualize_layer_diffs(layer_diffs):
    num_layers = len(layer_diffs)
    num_components = len(layer_diffs[0])
    
    fig, axs = plt.subplots(1, num_components, figsize=(24, 8))
    fig.suptitle(f"{long_context_model_name} <> {base_model_name}", fontsize=16)
    
    for i, component in enumerate(layer_diffs[0].keys()):
        component_diffs = [[layer_diff[component]] for layer_diff in layer_diffs]
        sns.heatmap(component_diffs, annot=True, fmt=".6f", cmap="YlGnBu", ax=axs[i], cbar_kws={"shrink": 0.8})
        axs[i].set_title(component)
        axs[i].set_xlabel("Layer")
        axs[i].set_ylabel("Difference")
        axs[i].set_xticks([])
        axs[i].set_yticks(range(num_layers))
        axs[i].set_yticklabels(range(num_layers))
        axs[i].invert_yaxis()
    
    plt.tight_layout()
    plt.show()

# 레이어별 가중치 차이 계산
layer_diffs = calculate_layer_diffs(long_context_model, base_model)

# target 모델 생성 및 diff 적용
target_model = AutoModelForCausalLM.from_pretrained(long_context_model_name)
apply_layer_diffs(target_model, layer_diffs)

# target 모델 저장
target_model.save_pretrained('./ctx_done')

# 가중치 차이 히트맵으로 시각화
visualize_layer_diffs(layer_diffs)